# Explore here

## Merge 1

In [1]:
import pandas as pd

# Rutas
path_df1    = "/Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/dataset_completo.csv"
path_vuelos = "/Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/merged_flight_dataset.csv"

# Parámetros
fecha_col    = "fecha"
fecha_umbral = pd.Timestamp("2022-01-01")
N            = 240_000

# 1) Cargo y filtro por fecha
df = pd.read_csv(path_df1, parse_dates=[fecha_col])
df = df[df[fecha_col] >= fecha_umbral].copy()
df["ciudad"] = df["ciudad"].str.lower().str.strip()

# 2) Merge con vuelos y renombrar precio
df_v = pd.read_csv(path_vuelos)
df_v["city_only"] = (
    df_v["destination_city"].astype(str)
         .str.rsplit(pat=" ", n=1).str[0]
         .str.lower().str.strip()
)
if "price" in df_v.columns:
    df_v = df_v.rename(columns={"price": "flight_price"})
else:
    price_col = [c for c in df_v.columns if "price" in c.lower()][0]
    df_v = df_v.rename(columns={price_col: "flight_price"})

m1 = pd.merge(
    df,
    df_v.drop(columns=["destination_city"]),
    how="left",
    left_on="ciudad",
    right_on="city_only"
).drop(columns=["city_only"])

# 3) Ordenar de más reciente a más antiguo
m1 = m1.sort_values(fecha_col, ascending=False)

# 4) Muestreo estratificado
counts     = m1["ciudad"].value_counts()
n_per_city = (counts / counts.sum() * N).round().astype(int).clip(lower=1)

def top_n(g):
    return g.head(n_per_city[g.name])

sample1 = m1.groupby("ciudad", group_keys=False).apply(top_n)

# Ajuste final
if len(sample1) > N:
    sample1 = sample1.head(N)
elif len(sample1) < N:
    extra    = m1.drop(sample1.index).head(N - len(sample1))
    sample1  = pd.concat([sample1, extra], ignore_index=True)

# 5) Guardar intermedio
out_inter   = "/Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/merged_dataset_vuelos_240k.csv"
out_inter_gz= out_inter + ".gz"
sample1.to_csv(out_inter, index=False)
sample1.to_csv(out_inter_gz, index=False, compression="gzip")

print(f"✅ Paso 1 completo: {len(sample1)} filas guardadas en:\n • {out_inter}\n • {out_inter_gz}")

/var/folders/jl/x5zpkzkd5m313zdg9lyh1f4h0000gn/T/ipykernel_31222/1876044743.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample1 = m1.groupby("ciudad", group_keys=False).apply(top_n)


✅ Paso 1 completo: 240000 filas guardadas en:
 • /Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/merged_dataset_vuelos_240k.csv
 • /Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/merged_dataset_vuelos_240k.csv.gz


## Merge 2

In [1]:
import pandas as pd

# Rutas completas
path_inter   = "/Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/merged_dataset_vuelos_240k.csv"
path_hoteles = "/Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/hotels_with_distance_dataset.csv"
out_final    = "/Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/total_data_240k.csv"
out_final_gz = out_final + ".gz"

# 1) Cargo la muestra intermedia (ya tiene vuelos)
df_int = pd.read_csv(path_inter, parse_dates=["fecha"])
df_int["ciudad"] = df_int["ciudad"].str.lower().str.strip()

# 2) Cargo hoteles con separador ';'
df_h = pd.read_csv(path_hoteles, sep=';')

# 3) Extraigo solo el nombre de la ciudad quitando el código de aeropuerto
df_h["city_only"] = (
    df_h["destination_city"]
      .astype(str)
      .str.rsplit(pat=" ", n=1).str[0]
      .str.lower()
      .str.strip()
)

# 4) Colapso a un solo hotel por ciudad: el más cercano al centro
df_h_nearest = (
    df_h.sort_values("distance_to_city_center_km", ascending=True)
        .drop_duplicates(subset="city_only", keep="first")
        .drop(columns=["destination_city"])  # ya no necesitamos la original
)

# 5) Merge final: left join para añadir datos de hotel a cada viaje
final = pd.merge(
    df_int,
    df_h_nearest,
    how="left",
    left_on="ciudad",
    right_on="city_only"
).drop(columns=["city_only"])

# 6) Guardar CSV final y GZIP
final.to_csv(out_final, index=False)
final.to_csv(out_final_gz, index=False, compression="gzip")

print(f"✅ Merge con hoteles completado: {len(final)} filas guardadas en:\n • {out_final}\n • {out_final_gz}")

✅ Merge con hoteles completado: 240000 filas guardadas en:
 • /Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/total_data_240k.csv
 • /Users/joanafernandes/Data Science Bootcamp/Predicccion_agencia_de_viajes_proyecto/data/processed/total_data_240k.csv.gz
